In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 57.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.8 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=48fbb45a69f35372004c80e899cbb846d984be44d28a5f7e9a86edb85a8490dc
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

#질의응답 모델 학습하기

모델 환경 설정

In [ ]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name = 'beomi/kcbert-base',
    downstream_corpus_name = 'korquad-v1',
    downstream_model_dir = '/gdrive/MyDrive/nlpbook/checkpoint-qa',
    max_seq_length = 128,
    max_query_length = 32,
    doc_stride = 64,
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate = 5e-5,
    epochs = 2,
    tpu_cores = 0 if torch.cuda.is_available() else 8,
    seed =7
)

In [ ]:
#랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

#로거 설정
nlpbook.set_logger(args)

#말뭉치 다운로드
nlpbook.download_downstream_dataset(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=4, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=2, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)
INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=4, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=2,

set seed: 7


토크나이저 준비

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

학습 데이터 구축

In [ ]:
#학습 데이터셋 구축
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'train'
)

#학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement = False),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_train_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 13.325 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_train_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 13.325 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_train_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 13.325 s]


평가용 데이터 구축

In [ ]:
#평가용 데이터셋 구축
from torch.utils.data import SequentialSampler
val_dataset = QADataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'val'
)

#평가용 데이터 로더 구축
val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_val_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 2.206 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_val_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 2.206 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_val_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 2.206 s]


모델 초기화

In [ ]:
from transformers import BertConfig, BertForQuestionAnswering
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name
)
model = BertForQuestionAnswering.from_pretrained(
    args.pretrained_model_name,
    config = pretrained_model_config
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

모델 학습시키기

In [ ]:
#Task 정의
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [ ]:
#Trainer 정의
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
#학습 개시
trainer.fit(
    task,
    train_dataloaders = train_dataloader,
    val_dataloaders = val_dataloader
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.318   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# 학습 마친 모델을 실전 투입하기

인퍼런스 설정

In [8]:
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/MyDrive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /gdrive/MyDrive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.46.ckpt


토크나이저 로드

In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

체크포인트 로드

In [12]:
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu')
)

In [14]:
#BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

#BERT 모델 초기화
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering(pretrained_model_config)

#체크포인트 주입
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

#평가 모드로 전환
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

인퍼런스 함수

In [15]:
def inference_fn(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [16]:
question='올해 연간 성장률에 대한 한은 전망치는?'
context='올해 연간 성장률이 1.1%로 한은 전망치 1.7%보다 크게 낮을 것으로 전망했다.'

inference_fn(question, context)

{'question': '올해 연간 성장률에 대한 한은 전망치는?',
 'context': '올해 연간 성장률이 1.1%로 한은 전망치 1.7%보다 크게 낮을 것으로 전망했다.',
 'answer': '1. 7 %'}

# 웹서비스

웹서비스 만들기 준비

In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken:  2KTww7XOEkN663tDzp1fRnphA2E_5x8UAQKw4GAhq3Bzp9t4m" > /root/.ngrok2/ngrok.yml

웹서비스 개시

In [ ]:
from ratsnlp.nlpbook.paircls import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()